In [ ]:
from syft_rds.orchestra import setup_rds_stack
from syft_rds import RDS_NOTEBOOKS_PATH

In [ ]:
from syft_core.url import SyftBoxURL

SyftBoxURL

In [ ]:
key = "full_flow_notebook_test"
stack = setup_rds_stack(key=key, log_level="INFO")

In [ ]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [ ]:
CWD = RDS_NOTEBOOKS_PATH / "quickstart"

In [ ]:
private_dir = CWD / "data" / "dataset-1" / "private"
mock_dir = CWD / "data" / "dataset-1" / "mock"
markdown_path = CWD / "data" / "dataset-1" / "description.md"

private_dir.mkdir(parents=True, exist_ok=True)
mock_dir.mkdir(parents=True, exist_ok=True)


with open(private_dir / "data.csv", "w") as f:
    f.write("-1,-2,-3")

with open(mock_dir / "data.csv", "w") as f:
    f.write("1,2,3")

with open(markdown_path, "w") as f:
    f.write("some description")

In [ ]:
try:
    data = do_client.dataset.create(
        name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
        path=private_dir,  # MUST EXIST
        mock_path=mock_dir,
        summary="dummy data",
        description_path=markdown_path,
    )
except Exception as e:
    if "already exists" in str(e):
        print(e)
    else:
        raise e

In [ ]:
do_client.dataset.get_all()

# Use dataset as DS

In [ ]:
ds_client.dataset.get_all()

In [ ]:
ds_client.dataset.get(name="dataset-1").get_mock_path()

In [ ]:
ds_client.dataset.get(name="dataset-1").get_private_path()

In [ ]:
dataset = ds_client.dataset.get(name="dataset-1")
dataset.describe()

In [ ]:
my_file = CWD / "data" / "code1.py"

In [ ]:
%%writefile {my_file}

import os

DATA_DIR = os.environ["DATA_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]

print("Hello, world!")

with open(os.path.join(OUTPUT_DIR, "output.txt"), "w") as f:
    f.write("ABC")


# Submit a job to create UserCode and a Job

In [ ]:
job = ds_client.job.submit(user_code_path=my_file, dataset_name=dataset.name)
job.describe()

In [ ]:
# Code file is moved to a location inside syftbox

code = job.user_code
assert code.local_dir.exists()

# Same as: job.code.describe()
job.show_user_code()

# DO reviews

In [ ]:
jobs = do_client.job.get_all(status="pending_code_review")
jobs

In [ ]:
job = jobs[-1]

# same as job.code.describe()
job.show_user_code()

In [ ]:
res_job = do_client.run_private(job)

In [ ]:
do_client.job.share_results(res_job);

# DS views the results

In [ ]:
job = ds_client.job.get_all()[-1]

In [ ]:
# DS can see files in the output path
job.describe()

In [ ]:
output_files = [f for f in job.output_path.glob("**/*") if f.is_file()]
assert len(output_files)

## Clean up

In [ ]:
from syft_rds.orchestra import remove_rds_stack_dir

remove_rds_stack_dir(key=key)